# Moviepy

In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# 포즈 감지 모델 초기화
mp_pose = mp.solutions.pose
pose_video = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [3]:
# 포즈 검출 함수
def detectPose(image_pose, pose, draw=False, display=False):
    
    original_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    
    resultant = pose.process(image_in_RGB)

    if resultant.pose_landmarks and draw:    

        mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=3, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))

    if display:
            
            plt.figure(figsize=[22,22])
            plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            plt.subplot(122);plt.imshow(original_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');

    else:
        
        return original_image, resultant

In [4]:
def norm(data):
    data = np.array(data)
    x = data.T[0]
    y = data.T[1]
    z = data.T[2]
    x_norm = (x - min(x)) / (max(x) - min(x))
    y_norm = (y - min(y)) / (max(y) - min(y))
    z_norm = (z - min(z)) / (max(z) - min(z))
    
    return (x_norm.tolist(), y_norm.tolist(), z_norm.tolist())

In [5]:
def link_vector(land):
    link_keypoint = [(0, 1),
        (1, 3),
        (3,	5),
        (5,	7),
        (5,	9),
        (5,	11),
        (1, 13),
        (13, 15),
        (15, 17),
        (17, 19),
        (17, 21),
        (0, 2),
        (2, 4),
        (4, 6),
        (4, 8),
        (4, 10),
        (4, 12),
        (2, 14),
        (14, 16),
        (16, 18),
        (18, 20),
        (18, 22)]
    
    a = []
    for link in link_keypoint:
        x = land[0][link[0]] - land[0][link[1]]
        y = land[1][link[0]] - land[1][link[1]]
        z = land[2][link[0]] - land[2][link[1]]
        a.append((x, y, z))
    return a

In [6]:
def angle_vector(land):
    
    import math
    
    angle_keypoint=[
        (0, 1, 3),
        (1, 3, 5),
        (3, 5, 9),
        (1, 13, 15),
        (13, 15, 17),
        (15, 17, 19),
        (15, 17, 21),
        (0, 2, 4),
        (2, 4, 6),
        (4, 6, 10),
        (2, 14, 16),
        (14, 16, 18),
        (16, 18, 20),
        (16, 18, 22)]
    
    a = []
    for angle in angle_keypoint:
        x = np.array([land[0][angle[0]] - land[0][angle[1]], land[1][angle[0]] - land[1][angle[1]], land[2][angle[0]] - land[2][angle[1]]])
        y = np.array([land[0][angle[1]] - land[0][angle[2]], land[1][angle[1]] - land[1][angle[2]], land[2][angle[1]] - land[2][angle[2]]])
        
        분자 = np.dot(x, y)
        분모 = np.sqrt(x.dot(x)) * np.sqrt(x.dot(x))
        try:
            a.append(math.acos(분자 / 분모))
        except:
            a.append(0)
    return (a)

In [7]:
def pose_feature(link, angle):
    산술평균_링크 = [np.mean(link.T[0]), np.mean(link.T[1]), np.mean(link.T[2])]
    표준편차_링크 = [np.std(link.T[0]), np.std(link.T[1]), np.std(link.T[2])]
    제곱평균_링크 = [np.mean(link.T[0]**2), np.mean(link.T[1]**2), np.mean(link.T[2]**2)]
    
    산술평균_앵글 = np.mean(angle)
    표준편차_앵글 = np.std(angle)
    제곱평균_앵글 = np.mean(angle)
    
    return(산술평균_링크+표준편차_링크+제곱평균_링크+[산술평균_앵글]+[표준편차_앵글]+[제곱평균_앵글])

In [8]:
import joblib
model = joblib.load('./RandomForest.pkl')

c:\ProgramData\Miniconda3\envs\kdig\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\ProgramData\Miniconda3\envs\kdig\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
def get_sim(target, user):
    target_landmarks = target.pose_world_landmarks.landmark
    target_lm = [(i.x, i.y, i.z) for num, i in enumerate(target_landmarks) if num not in range(1, 11)]
    target_norm = norm(target_lm)
    target_link_vector = link_vector(target_norm)
    target_angle_vector = angle_vector(target_norm)
    
    user_landmarks = user.pose_world_landmarks.landmark
    user_lm = [(i.x, i.y, i.z) for num, i in enumerate(user_landmarks) if num not in range(1, 11)]
    user_norm = norm(user_lm)
    user_link_vector = link_vector(user_norm)
    user_angle_vector = angle_vector(user_norm)
    
    link_diff = np.array(target_link_vector) - user_link_vector
    angle_diff = np.array(target_angle_vector) - user_angle_vector
    feature = pose_feature(link_diff, angle_diff); feature = np.array(feature).reshape(1, -1)
    similarity = model.predict_proba(feature)[0][1]
    
    return similarity

## BTS - Dynamite

In [10]:
from moviepy.editor import *
video_clip = VideoFileClip('./dance/BTS Dynamite_target.mp4')
video_len = video_clip.duration
video_clip2 = VideoFileClip('./dance/BTS Dynamite_user.mp4').subclip(0.2, video_len+0.2)

In [11]:
audioclip = video_clip.audio
audioclip.write_audiofile('./target.mp3')

MoviePy - Writing audio in ./target.mp3


MoviePy - Done.


In [12]:
w = 640
h = 720
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.avi', fourcc, 1/0.03, (w, h))

ss = []
text = ''

for num, i in enumerate(np.arange(0, video_len, 0.03)):
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    img_taget = cv2.resize(img, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    img_user = cv2.resize(img2, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    
    if (num+1) % 20 == 0:
        ss_mean = round(np.mean([i*100 for i in ss[num-18:] if i is not None]))
        score = ('BAD' if ss_mean <= 30 else 'GOOD' if ss_mean <= 60 else 'PERPECT')
        text = f'{score}'

    try:
        result = detectPose(img_taget, pose_video)[1]
        result2 = detectPose(img_user, pose_video)[1]
        
        similarity = get_sim(result, result2)
        ss.append(similarity)

        numpy_vertical = np.vstack((img_taget, img_user))
        numpy_vertical_concat = np.concatenate((img_taget, img_user), axis=0)
        
        ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
        
        cv2.putText(ver_cv,  text, (10, 380), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", ver_cv)
        
    except:
        numpy_vertical = np.vstack((img_taget, img_user))
        numpy_vertical_concat = np.concatenate((img_taget, img_user), axis=0)
        ss.append(None)
        
        ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
        cv2.putText(ver_cv,  text, (10, 380), cv2.FONT_HERSHEY_DUPLEX, 2, (199, 114, 255), 2, cv2.LINE_AA)        # BGR
        cv2.imshow("target", ver_cv)

    out.write(ver_cv) #프레임 쓰기
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
out.release()

In [ ]:
total = round(np.mean(ss)*100)
print("최종 점수: ", total)

if total <= 30:
    print("FAIL")
elif total <= 40:
    print("BAD")
elif total <=60:
    print("OK")
elif total <=70:
    print("GOOD")
else:
    print("GREAT")

최종 점수:  41
OK


In [ ]:
w = 640
h = 720
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('test.avi', fourcc, 1/0.01, (w, h))

for i in np.arange(0, 84.2, 0.01):
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    img_taget = cv2.resize(img, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    img_user = cv2.resize(img2, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    numpy_vertical = np.vstack((img_taget, img_user))
    numpy_vertical_concat = np.concatenate((img_taget, img_user), axis=0)
    
    ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
    cv2.imshow("target", ver_cv)
    out.write(ver_cv) #프레임 쓰기
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
out.release()

In [ ]:
from moviepy.editor import *
videoclip = VideoFileClip("output.avi")
audioclip = audioclip

videoclip.audio = audioclip
videoclip.write_videofile("new video.mp4")

Moviepy - Building video new video.mp4.
MoviePy - Writing audio in new videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video new video.mp4



Moviepy - Done !
Moviepy - video ready new video.mp4


## I-DLE - TOMBOY

In [ ]:
from moviepy.editor import *
video_clip = VideoFileClip('./영상/I-DLE TOMBOY.mp4').subclip(4.3, 83.48)
video_clip2 = VideoFileClip('./영상/I-DLE TOMBOY_user.mp4').subclip(0.7, 74.9)

In [ ]:
# 싱크 맞는지 확인
for i in np.arange(0, 84.2, 0.03):
   
    img = video_clip.get_frame(i)
    img2 = video_clip2.get_frame(i)
    
    img_taget = cv2.resize(img, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    img_user = cv2.resize(img2, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    numpy_vertical = np.vstack((img_taget, img_user))
    numpy_vertical_concat = np.concatenate((img_taget, img_user), axis=0)
    
    ver_cv = cv2.cvtColor(numpy_vertical, cv2.COLOR_BGR2RGB)
    cv2.imshow("target", ver_cv)
        
    # out.write(ver_cv) #프레임 쓰기
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

# 웹캠 녹화

In [ ]:
import cv2
import sys
import time
# 노트북 웹캠에서 받아오는 영상을 저장하기

# 기본 카메라 객체 생성
cap = cv2.VideoCapture(0)

# 열렸는지 확인
if not cap.isOpened():
    print("Camera open failed!")
    sys.exit()

# 웹캠의 속성 값을 받아오기
# 정수 형태로 변환하기 위해 round
w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) # 카메라에 따라 값이 정상적, 비정상적

if fps ==0 :
    fps=30

# fourcc 값 받아오기, *는 문자를 풀어쓰는 방식, *'DIVX' == 'D', 'I', 'V', 'X'
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

# 1프레임과 다음 프레임 사이의 간격 설정
delay = round(1000/fps)

# 웹캠으로 찰영한 영상을 저장하기
# cv2.VideoWriter 객체 생성, 기존에 받아온 속성값 입력
out = cv2.VideoWriter('1.avi', fourcc, fps, (w, h))

begins = []
# 제대로 열렸는지 확인
if not out.isOpened():
    print('File open failed!')
    cap.release()
    sys.exit()
    
# 프레임을 받아와서 저장하기
while True:                 # 무한 루프
    ret, frame = cap.read() # 카메라의 ret, frame 값 받아오기

    if not ret:             #ret이 False면 중지
        break
    
    cv2.putText(frame, text=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), org=(30, 450), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0), thickness=2)
    cv2.imshow('frame', frame)
    begins.append(time.time())
    out.write(frame) # 영상 데이터만 저장. 소리는 X
    
    # print(cap.get(cv2.CAP_PROP_FPS))
    if cv2.waitKey(delay) == 27: # esc를 누르면 강제 종료
        end = time.time()
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# 카운트다운 효과

In [ ]:
import cv2
import time
import numpy as np

In [ ]:
for i in range(3, 0, -1):
    zeros = np.zeros((720, 1280), dtype=np.uint8)
    cv2.putText(zeros, str(i), (580, 360), cv2.FONT_HERSHEY_SIMPLEX, 10, (255,255,255), 20, cv2.LINE_AA)
    cv2.imshow('test', zeros)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()

In [ ]:
import cv2

# 웹캠 연결
cap = cv2.VideoCapture('./영상/I-DLE TOMBOY.mp4')


while True:

    # 웹캠에서 이미지 읽어옴
    ret,img_color = cap.read()

    if ret == False:
        print('웹캠에서 영상을 읽을 수 없습니다.')
        break
    
    # print(cap.get(cv2.CAP_PROP_POS_FRAMES))
    
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == 1:
        for i in range(3, 0, -1):
            zeros = np.zeros((720, 1280), dtype=np.uint8)
            cv2.putText(zeros, str(i), (580, 360), cv2.FONT_HERSHEY_SIMPLEX, 10, (255,255,255), 20, cv2.LINE_AA)
            cv2.imshow('test', zeros)
            cv2.waitKey(1000)
            cv2.destroyAllWindows()
        
    cv2.imshow("test", img_color)


    # ESC키 누르면 중지
    if cv2.waitKey(1)&0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.get(cv2.CAP_PROP_POS_FRAMES) == 0

True